In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [2]:
%%hive
DROP TABLE IF EXISTS tbl0;
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
--
DROP TABLE IF EXISTS tbl1;
CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1;

DROP TABLE IF EXISTS tbl0;
OK
Time taken: 13.178 seconds
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 1.167 seconds
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
Loading data to table default.tbl0
OK
Time taken: 2.44 seconds
--
DROP TABLE IF EXISTS tbl1;
OK
Time taken: 0.01 seconds
CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 0.084 seconds
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1;
Loading data to table default.tbl1
OK
Time taken: 1.274 seconds


In [3]:
%%hive 
SELECT * FROM tbl0;

SELECT * FROM tbl0;
OK
1	D	4	2016-06-25	["a","e","c","d"]	{"bb":10,"dd":20,"cc":40}
2	C	4	2015-05-14	["a","c"]	{"dd":40,"bb":20,"cc":10}
3	D	6	2014-12-26	["b","d"]	{"aa":10,"bb":40}
4	D	5	2016-06-25	["a","c","e","b","d"]	{"bb":40,"dd":20,"aa":10,"cc":30}
5	C	7	2016-05-23	["d","e","c"]	{"cc":10,"aa":20}
6	A	2	2018-06-14	["a","d"]	{"aa":20}
7	B	3	2014-12-22	["a","e","d"]	{"cc":20}
8	C	6	2015-08-20	["d","a","c","e"]	{"cc":10,"aa":20}
9	B	3	2017-12-08	["b","a","c","e"]	{"cc":40,"dd":10,"aa":30,"bb":20}
10	B	7	2015-07-28	["c","d","e","a","b"]	{"aa":10,"dd":40,"cc":30}
Time taken: 2.28 seconds, Fetched: 10 row(s)


In [79]:
%%hive 
DROP TABLE IF EXISTS dataTable;
CREATE TABLE dataTable
AS
    SELECT c2, collect_set(c1)
    FROM
        tbl0
GROUP BY
    c2;

DROP TABLE IF EXISTS dataTable;
OK
Time taken: 0.668 seconds
CREATE TABLE dataTable
AS
    SELECT c2, collect_set(c1)
    FROM
        tbl0
GROUP BY
    c2;
Query ID = root_20200228115959_6b9445cf-03e7-4498-9bed-122432030e88
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1582886269318_0013, Tracking URL = http://7af61c813230:8088/proxy/application_1582886269318_0013/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1582886269318_0013
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-02-28 12:00:06,562 Stage-1 map = 0%,  reduce = 0%
2020-02-28 12:00:12,087 Stage-1 map = 100%,  re

In [81]:
%%hive 
SELECT * FROM dataTable;

SELECT * FROM dataTable;
OK
A	[6]
B	[7,9,10]
C	[2,5,8]
D	[1,3,4]
Time taken: 0.196 seconds, Fetched: 4 row(s)


In [91]:
%hive_quit

In [92]:
## Copia Archivos a sistema HDFS
##
## Se usan un directorio temporal en el HDFS. La siguiente
## instrucción muestra el contenido del dicho directorio
##
!hdfs dfs -ls /tmp

Found 4 items
drwxrwx---   - root supergroup          0 2020-02-28 10:38 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-28 10:51 /tmp/hive
drwxrwxrwx   - root supergroup          0 2020-02-28 11:38 /tmp/output
drwxr-xr-x   - root supergroup          0 2020-02-28 11:35 /tmp/wordcount


In [93]:
## Crea la carpeta wordcount en el hdfs
##
!hdfs dfs -mkdir /tmp/wordcount

mkdir: `/tmp/wordcount': File exists


In [94]:
## Verifica la creación de la carpeta
##
!hdfs dfs -ls /tmp/

Found 4 items
drwxrwx---   - root supergroup          0 2020-02-28 10:38 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-28 10:51 /tmp/hive
drwxrwxrwx   - root supergroup          0 2020-02-28 11:38 /tmp/output
drwxr-xr-x   - root supergroup          0 2020-02-28 11:35 /tmp/wordcount


In [95]:
## Copia los archvios del directorio local wordcount/
## al directorio /tmp/wordcount/ en el hdfs
##
!hdfs dfs -copyFromLocal 'data.tsv'  /tmp/wordcount/

copyFromLocal: `data.tsv': No such file or directory


In [96]:
##
## Verifica que los archivos esten copiados
## en el hdfs
##
!hdfs dfs -ls /tmp/wordcount

In [97]:
####################################################################################################################
## Generación del script y ajuste del código

In [102]:
%%writefile wordcount.hql

DROP TABLE IF EXISTS tbl0;
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
--
DROP TABLE IF EXISTS tbl1;
CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1;

DROP TABLE IF EXISTS dataTable;
CREATE TABLE dataTable
AS
    SELECT c2, collect_set(c1)
    FROM
        tbl0
GROUP BY
    c2;
    
INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'

SELECT * FROM dataTable;

Overwriting wordcount.hql


In [103]:
## Ejecucion
!hive -S -e 'source wordcount.hql'

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


In [104]:
## Copia de los resultados a la maquina local (vagrant)
!hadoop fs -copyToLocal /tmp/output output
!ls output/*

output/000000_0

output/output:
000000_0


In [105]:
!cat output/000000_0 | head

A,6
B,7910
C,258
D,134
